# Segmenting and Clustering Neighborhoods in Toronto - Homework

## We install and import the packages and libraries required.

In [32]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json  

%pip install geopy 
from geopy.geocoders import Nominatim  

%pip install requests
import requests  
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt 

%matplotlib inline 


%pip install sklearn
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs

%pip install folium  
import folium 

%pip install lxml
import lxml.html as lh
import random 
%lsmagic
from IPython.display import Image

print('Libraries imported.')

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Libraries imported.


In [33]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [34]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [35]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()[:-1]
    print ('%d: %s'%(i,name))
    col.append((name,[]))

1: Postal Code
2: Borough
3: Neighbourhood


In [36]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content()[:-1] 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [37]:
[len(C) for (title,C) in col]

[181, 181, 181]

In [38]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [39]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [40]:
df=df.drop(df[df["Borough"]=="Not assigned"].index)
df=df.drop(df[df["Borough"]=="Canadian postal codes"].index)
df.reset_index(drop=True,inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [41]:
df.shape

(103, 3)

In [42]:
TorontoGeo='http://cocl.us/Geospatial_data'
dfPoCoLaLo=pd.read_csv(TorontoGeo)
dfPoCoLaLo=dfPoCoLaLo.rename(columns={'Postal Code':'PostalCode'})
df=df.rename(columns={'Postal Code':'PostalCode'})
df=df.rename(columns={'Neighbourhood':'Neighborhood'})
dfPoCoLaLo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [43]:
# We integrate the columns Latitude and Longitude with the data frame of neighborhoods of Toronto. 
df['Latitude']=None
df['Longitude']=None
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,None,None
1,M4A,North York,Victoria Village,None,None
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",None,None
3,M6A,North York,"Lawrence Manor, Lawrence Heights",None,None
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",None,None


In [44]:
# Now assigned the Latitude-Longitude information to each neighborhood according your postal code.
for i in df.index:
    for j in dfPoCoLaLo.index:
        if (df['PostalCode'][i]==dfPoCoLaLo['PostalCode'][j]):
            df['Latitude'][i]=dfPoCoLaLo['Latitude'][j]
            df['Longitude'][i]=dfPoCoLaLo['Longitude'][j]
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6679,-79.5322
6,M1B,Scarborough,"Malvern, Rouge",43.8067,-79.1944
7,M3B,North York,Don Mills,43.7459,-79.3522
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7064,-79.3099
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789


In [45]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [46]:
address = 'Toronto City, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [47]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
Now, for our analisys, we select the boroughs with name xx-York (for example, North York, East York, ...), and we created with them the data frame df_xxyork.

In [18]:
df_xxyork=df.iloc[[0,1,3,7,8,10,13,14,16,21,23,27,28,29,33,34,35,39,40,45,46,49,50,52,53,55,56,57,59,60,63,64,66,72],:]
df_xxyork

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
7,M3B,North York,Don Mills,43.7459,-79.3522
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7064,-79.3099
10,M6B,North York,Glencairn,43.7096,-79.4451
13,M3C,North York,Don Mills,43.7259,-79.3409
14,M4C,East York,Woodbine Heights,43.6953,-79.3184
16,M6C,York,Humewood-Cedarvale,43.6938,-79.4282
21,M6E,York,Caledonia-Fairbanks,43.689,-79.4535


In [19]:
df_xxyork.reset_index(drop=True,inplace=True)
df_xxyork.index=df_xxyork.index.astype(int)
df_xxyork

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
3,M3B,North York,Don Mills,43.7459,-79.3522
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7064,-79.3099
5,M6B,North York,Glencairn,43.7096,-79.4451
6,M3C,North York,Don Mills,43.7259,-79.3409
7,M4C,East York,Woodbine Heights,43.6953,-79.3184
8,M6C,York,Humewood-Cedarvale,43.6938,-79.4282
9,M6E,York,Caledonia-Fairbanks,43.689,-79.4535


In [52]:
df_xxyork.iloc[6,2]="Don Mills 1"
df_xxyork.iloc[20,2]="Downsview 1"
df_xxyork.iloc[24,2]="Downsview 2"
df_xxyork.iloc[29,2]="Downsview 3"
df_xxyork

c:\users\home\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-52-1953f3af16bc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xxyork.iloc[6,2]="Don Mills 1"
<ipython-input-52-1953f3af16bc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xxyork.iloc[20,2]="Downsview 1"
<ipython-input-5

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
3,M3B,North York,Don Mills,43.7459,-79.3522
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7064,-79.3099
5,M6B,North York,Glencairn,43.7096,-79.4451
6,M3C,North York,Don Mills 1,43.7259,-79.3409
7,M4C,East York,Woodbine Heights,43.6953,-79.3184
8,M6C,York,Humewood-Cedarvale,43.6938,-79.4282
9,M6E,York,Caledonia-Fairbanks,43.689,-79.4535


In [53]:
address = 'North York, TO'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [54]:
# create map of Etobicoke using latitude and longitude values
map_xxyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_xxyork['Latitude'], df_xxyork['Longitude'], df_xxyork['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_xxyork)  
    
map_xxyork

In [55]:
CLIENT_ID = 'TS0FSEPKAPDDVEEQIKLTVL1TFQLJY2FTKORIMB1U0UDARQVC' # your Foursquare ID
CLIENT_SECRET = 'ENJ1RZ5JBZJTFQSIBJP5NY5EUYMT5OVFN2Q1055TWOCKTDOX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TS0FSEPKAPDDVEEQIKLTVL1TFQLJY2FTKORIMB1U0UDARQVC
CLIENT_SECRET:ENJ1RZ5JBZJTFQSIBJP5NY5EUYMT5OVFN2Q1055TWOCKTDOX


In [56]:
df.loc[0, 'Neighborhood']

'Parkwoods'

In [57]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [25]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=TS0FSEPKAPDDVEEQIKLTVL1TFQLJY2FTKORIMB1U0UDARQVC&client_secret=ENJ1RZ5JBZJTFQSIBJP5NY5EUYMT5OVFN2Q1055TWOCKTDOX&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [58]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1faca05dc8d450a1605d01'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [59]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [60]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-60-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [61]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


# **2. Explore Neighborhoods in xxyork** 

## Let's create a function to repeat the same process to all the neighborhoods in xxyork

In [62]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [63]:
xxyork_venues = getNearbyVenues(names=df_xxyork['Neighborhood'],
                                   latitudes=df_xxyork['Latitude'],
                                   longitudes=df_xxyork['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Parkview Hill, Woodbine Gardens
Glencairn
Don Mills 1
Woodbine Heights
Humewood-Cedarvale
Caledonia-Fairbanks
Leaside
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Bayview Village
Downsview
York Mills, Silver Hills
Downsview 1
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview 2
Bedford Park, Lawrence Manor East
Del Ray, Mount Dennis, Keelsdale and Silverthorn
Humberlea, Emery
Willowdale, Willowdale East
Downsview 3
Runnymede, The Junction North
Weston
York Mills West
Willowdale, Willowdale West


In [64]:
# Print the number of row and columns of xxyork_venues
print(xxyork_venues.shape)
xxyork_venues.head()

(337, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [65]:
xxyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",28,28,28,28,28,28
Caledonia-Fairbanks,4,4,4,4,4,4
"Del Ray, Mount Dennis, Keelsdale and Silverthorn",4,4,4,4,4,4
Don Mills,6,6,6,6,6,6
Don Mills 1,20,20,20,20,20,20
Downsview,4,4,4,4,4,4
Downsview 1,4,4,4,4,4,4


In [66]:
print('There are {} uniques categories.'.format(len(xxyork_venues['Venue Category'].unique())))

There are 119 uniques categories.


# 3. Analyze Each Neighborhood


In [67]:
# one hot encoding
xxyork_onehot = pd.get_dummies(xxyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
xxyork_onehot['Neighborhood'] = xxyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [xxyork_onehot.columns[-1]] + list(xxyork_onehot.columns[:-1])
xxyork_onehot = xxyork_onehot[fixed_columns]

xxyork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Event Space,Fast Food Restaurant,Field,Fish & Chips Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hobby Shop,Hockey Arena,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lingerie Store,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Skating Rink,Smoothie Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Warehouse Store,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [68]:
xxyork_onehot.shape

(337, 120)

In [69]:
xxyork_grouped = xxyork_onehot.groupby('Neighborhood').mean().reset_index()
xxyork_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Electronics Store,Event Space,Fast Food Restaurant,Field,Fish & Chips Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hobby Shop,Hockey Arena,Hotel,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lingerie Store,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Skating Rink,Smoothie Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Warehouse Store,Women's Store,Yoga Studio
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.0,0.050000,0.000000,0.000000,0.00,0.05,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.0,0.05,0.000000,0.050000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.050000,0.000000,0.050000,0.000000,0.050000,0.000000,0.050000,0.050000,0.000000,0.00,0.0,0.000000,0.000000,0.050000,0.000000,0.050000,0.050000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.050000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.250000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.00,0.035714,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.035714,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.035714,0.0,0.035714,0.000000,0.00,0.000000,0.000000,0.071429,0.035714,0.000000,0.000000,0.000000,0.0

In [39]:
num_top_venues = 5

for hood in xxyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = xxyork_grouped[xxyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
         venue  freq
0  Coffee Shop  0.10
1         Bank  0.10
2  Bridal Shop  0.05
3  Supermarket  0.05
4         Park  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2  Japanese Restaurant  0.25
3   Chinese Restaurant  0.25
4       Massage Studio  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0          Restaurant  0.07
1  Italian Restaurant  0.07
2      Sandwich Place  0.07
3         Coffee Shop  0.07
4     Thai Restaurant  0.07


----Caledonia-Fairbanks----
           venue  freq
0           Park  0.50
1  Women's Store  0.25
2           Pool  0.25
3      Juice Bar  0.00
4  Movie Theater  0.00


----Del Ray, Mount Dennis, Keelsdale and Silverthorn----
               venue  freq
0        Coffee Shop  0.25
1                Bar  0.25
2     Sandwich Place  0.25
3     Discount Store  0.25
4  Accessories Store  0.00


----Don Mills--

In [70]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [71]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = xxyork_grouped['Neighborhood']

for ind in np.arange(xxyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(xxyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.index=neighborhoods_venues_sorted.index.astype(int)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Mobile Phone Shop,Diner,Middle Eastern Restaurant,Park,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop
1,Bayview Village,Japanese Restaurant,Chinese Restaurant,Café,Bank,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega
2,"Bedford Park, Lawrence Manor East",Restaurant,Italian Restaurant,Thai Restaurant,Sandwich Place,Coffee Shop,Greek Restaurant,Comfort Food Restaurant,Pub,Pizza Place,Pharmacy
3,Caledonia-Fairbanks,Park,Women's Store,Pool,Yoga Studio,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Discount Store,Restaurant,Bar,Sandwich Place,Yoga Studio,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop


# 4. Cluster Neighborhoods

## Run k-means to cluster the neighborhood into 5 clusters.


In [72]:
# set number of clusters
kclusters = 5

xxyork_grouped_clustering = xxyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(xxyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 4])

In [73]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

xxyork_merged = df_xxyork

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
xxyork_merged = xxyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
xxyork_merged.index=xxyork_merged.index.astype(int)
xxyork_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7533,-79.3297,0,Park,Food & Drink Shop,Discount Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice
1,M4A,North York,Victoria Village,43.7259,-79.3156,1,Coffee Shop,Pizza Place,French Restaurant,Hockey Arena,Portuguese Restaurant,Yoga Studio,Diner,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648,1,Clothing Store,Accessories Store,Sporting Goods Shop,Women's Store,Event Space,Coffee Shop,Miscellaneous Shop,Boutique,Furniture / Home Store,Vietnamese Restaurant
3,M3B,North York,Don Mills,43.7459,-79.3522,1,Japanese Restaurant,Gym,Caribbean Restaurant,Café,Athletics & Sports,Baseball Field,Department Store,Diner,Dim Sum Restaurant,Dessert Shop
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7064,-79.3099,1,Pizza Place,Gastropub,Athletics & Sports,Pharmacy,Breakfast Spot,Bank,Intersection,Gym / Fitness Center,Food Court,Construction & Landscaping


In [74]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(xxyork_merged['Latitude'], xxyork_merged['Longitude'], xxyork_merged['Neighborhood'], xxyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 5. Examine Clusters

## Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

## Cluster 1

In [75]:
xxyork_merged.loc[xxyork_merged['Cluster Labels'] == 0, xxyork_merged.columns[[1] + list(range(5, xxyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Park,Food & Drink Shop,Discount Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice
9,York,0,Park,Women's Store,Pool,Yoga Studio,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
16,East York,0,Park,Convenience Store,Intersection,Discount Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Curling Ice
23,North York,0,Park,Golf Course,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega
32,North York,0,Park,Construction & Landscaping,Convenience Store,Golf Course,Clothing Store,Coffee Shop,Comfort Food Restaurant,Cosmetics Shop,Curling Ice,Deli / Bodega


## Cluster 2

In [76]:
xxyork_merged.loc[xxyork_merged['Cluster Labels'] == 1, xxyork_merged.columns[[1] + list(range(5, xxyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Coffee Shop,Pizza Place,French Restaurant,Hockey Arena,Portuguese Restaurant,Yoga Studio,Diner,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
2,North York,1,Clothing Store,Accessories Store,Sporting Goods Shop,Women's Store,Event Space,Coffee Shop,Miscellaneous Shop,Boutique,Furniture / Home Store,Vietnamese Restaurant
3,North York,1,Japanese Restaurant,Gym,Caribbean Restaurant,Café,Athletics & Sports,Baseball Field,Department Store,Diner,Dim Sum Restaurant,Dessert Shop
4,East York,1,Pizza Place,Gastropub,Athletics & Sports,Pharmacy,Breakfast Spot,Bank,Intersection,Gym / Fitness Center,Food Court,Construction & Landscaping
5,North York,1,Park,Pizza Place,Pub,Japanese Restaurant,Dim Sum Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
6,North York,1,Coffee Shop,Gym,Beer Store,Restaurant,Art Gallery,Clothing Store,Asian Restaurant,Discount Store,Sandwich Place,Japanese Restaurant
7,East York,1,Park,Pharmacy,Curling Ice,Beer Store,Skating Rink,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
8,York,1,Hockey Arena,Tennis Court,Trail,Field,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega,Curling Ice,Yoga Studio
10,East York,1,Coffee Shop,Sporting Goods Shop,Bank,Furniture / Home Store,Burger Joint,Liquor Store,Beer Store,Restaurant,Bike Shop,Brewery
11,North York,1,Golf Course,Athletics & Sports,Pool,Mediterranean Restaurant,Dog Run,French Restaurant,Food Truck,Comfort Food Restaurant,Construction & Landscaping,Convenience Store


## Cluster 3

In [77]:
xxyork_merged.loc[xxyork_merged['Cluster Labels'] == 2, xxyork_merged.columns[[1] + list(range(5, xxyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,North York,2,Cafeteria,Yoga Studio,Chocolate Shop,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega


## Cluster 4

In [78]:
xxyork_merged.loc[xxyork_merged['Cluster Labels'] == 3, xxyork_merged.columns[[1] + list(range(5, xxyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,York,3,Convenience Store,Yoga Studio,Distribution Center,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store


## Cluster 5

In [79]:
xxyork_merged.loc[xxyork_merged['Cluster Labels'] == 4, xxyork_merged.columns[[1] + list(range(5, xxyork_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,North York,4,Food Truck,Baseball Field,Yoga Studio,Distribution Center,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice
27,North York,4,Baseball Field,Yoga Studio,Distribution Center,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega
